In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import javalang
from dataclasses import dataclass
from tqdm.auto import tqdm
from pickle_cache import PickleCache
from javalang import tree
from pprint import pprint
import textwrap
import copy
from iterextras import par_for, unzip
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from utils import *
sns.set()

In [ ]:
solutions = get_solutions()

In [ ]:
# import traceback

# inlined = {}    
# fail = 0
# for student, solns in tqdm(list(solutions.items())):
#     try:
#         methods = get_methods(solns[-1])
#         start = copy.deepcopy(methods['run'])
#         inlined[student] = Inline(methods).visit(start).body
# #     except RecursionError:
# #         pass
#     except Exception:
#         #traceback.print_exc()
#         fail += 1

In [ ]:
grammars = {}
for k, solns in tqdm(list(solutions.items())):   
    try:
        methods = get_methods(solns[-1])
        if 'run' not in methods:
            continue

        generator = GrammarGenerator(k, methods)
        generator.generate(methods['run'])
        grammars[k] = Grammar(generator.productions)
    except (RecursionError, Unimplemented, IndexError):
        pass
    except Exception:
        print(k)
        raise

In [ ]:
productions = {k: p for g in grammars.values() for k, p in g.productions.items()}

productions['start'] = Production(rules=[
    Rule(
        parts=[f'{k}_run'],
        prob=1. / len(grammars)
    )
    for k in grammars.keys()
])
g = Grammar(productions=productions)
g, cs = g.simplify()

In [ ]:
p = g.sample()

In [ ]:
p

In [ ]:
# Find all 
groups = sorted(cs.items(), key=lambda t: len(t[1]))[::-1]
for k, group in groups[:50]:
    other_names = ['_'.join(k2.split('_')[1:]) for k2 in group if '_rule' not in k2]
    print(k, len(group), len(other_names), other_names)
    pprint(g.expand(k))
    print()

In [ ]:
# # Compute overlap in productions between students
# N = len(grammars)
# dists = np.zeros((N, N))
# index = list(grammars.keys())
# rev_index = {s: i for i, s in enumerate(index)}

# for k1, grammar1 in tqdm(list(grammars.items())):
#     for k2, grammar2 in grammars.items():
#         count = 0
#         grammar1, grammar2 = normalize_grammars(grammar1, grammar2)
#         for p1 in grammar1.productions.values():
#             for p2 in grammar2.productions.values():
#                 if p1 == p2:
#                     count += 1
#                     break
#         dists[rev_index[k1],rev_index[k2]] = count

# dists_sorted = np.dstack(np.unravel_index(np.argsort(dists, axis=None), dists.shape))[0,::-1]
# dists_sorted = dists_sorted[dists_sorted[:,0] != dists_sorted[:,1]]        